# Project 6 - Exotic Options and Car Loan Models


**Name:** Bowen Chen

**Section:** 1

**Date:** Feb 18, 2018 

In [ ]:
# python set up
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd

## Question 1

Consider a 12-month Fixed Strike Lookback Call and Put options, when the interest rate is 3% per annum, the current stock price is \$98 and the strike price is \$100. Use the MC simulation method to estimate the prices of Call and Put options for the following range of volatilities: from 12% to 48%, in increments of 4%. The payoff of the call option is $(S_{max} − X)^+$ and The payoff of the put option is $(X - S_{min})^+$ 

**Solution:**

Start with the Monte Carlo Simulation of stock prices, with half of the paths being antithetic paths

In [ ]:
def StockPrices(S0, r, sd, T, paths, steps):
    
    dt = T/steps
    
    # Generate stochastic process and its antithetic paths
    Z = np.random.normal(0, 1, paths//2 * steps).reshape((paths//2, steps))
    Z_inv = -Z
    
    dWt = math.sqrt(dt) * Z
    dWt_inv = math.sqrt(dt) * Z_inv
    
    # bind the normal and antithetic Wt
    dWt = np.concatenate((dWt, dWt_inv), axis=0)
    
    St = np.zeros((paths, steps + 1))
    St[:, 0] = S0
    
    for i in range (1, steps + 1):
        St[:, i] = St[:, i - 1]*np.exp((r - 1/2*np.power(sd, 2))*dt + sd*dWt[:, i - 1])
    
    return St[:, 1:]

Assume the lookback option is a European Option, now define the function that could find the value of the lookback call and lookback put

In [ ]:
def LookBackOption(S0, K, r, sd, T, paths, Type):
    
    # use daily frequency
    steps = 252
    St = StockPrices(S0, r, sd, T, paths, steps)
    
    # find the maximum and minimum stock price incurred for each path
    St_max = St.max(axis = 1)
    St_min = St.min(axis = 1)
    
    # find the lookback option value
    if Type == "call":
        option_value = np.exp(-r*T) * np.mean(np.maximum(St_max - K, 0))
    elif Type == "put":
        option_value = np.exp(-r*T) * np.mean(np.maximum(K - St_min, 0))
        
    return option_value

Initialize parameters

In [ ]:
np.random.seed(7)
r = 0.03
paths = 10000
K = 100
S0 = 98
T = 1
sd = np.arange(0.12, 0.49, 0.04)

Find values for lookback call and lookback put options

In [ ]:
call_value = [LookBackOption(S0, K, r, i, T, paths, "call") for i in sd]
put_value = [LookBackOption(S0, K, r, i, T, paths, "put") for i in sd]

Plot the lookback call and the lookback put values on the same plot

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(sd, call_value,  linestyle='--', 
         marker='o')

plt.xlabel('Volitality $\sigma$')
plt.ylabel('European Call Value $')
plt.title('European Lookback Call Option Values')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(sd, put_value,  linestyle='--', 
         marker='o', color = 'orange')
plt.xlabel('Volitality $\sigma$')
plt.ylabel('European Put Value $')
plt.title('European Lookback Put Option Values')
plt.show()

## Question 2

Assume that the value of a collateral follows a jump-diffusion process:

$$\frac{dV_t}{V_t} = \mu dt + \sigma dW_t + \gamma dJ_t$$

$V_0$ = \$20,000, $L_0$ = \$22,000, $\mu$ = −0.1, $\sigma$ = 0.2, $\gamma$ = −0.4, $\lambda_1$ = 0.2, T = 5 years, $r_0$ =
0.02, $\delta$ = 0.25, $\lambda_2$ = 0.4, $\alpha$ = 0.7, $\epsilon$ = 0.95

Simulate the loan collateral value as jump diffusion process using timeline method. Assume monthly time step

In [ ]:
def loanCollateral_Vt(V0, mu, sigma, gamma, lambda1, T, paths):
    
    dt = 1/12
    steps = T * 12

    # Generate stochastic process and its antithetic paths
    Z = np.random.normal(0, 1, paths * steps).reshape((paths, steps))
    dWt = math.sqrt(dt) * Z

    # Initialize Vt process
    Vt = np.zeros((paths, steps + 1))
    Vt[:, 0] = V0

    # Build Vt Process
    for i in range (1, steps + 1):
        Vt[:, i] = (Vt[:, i - 1] * np.exp((mu - 1/2*np.power(sigma, 2)) * dt + sigma * dWt[:, i - 1]) 
                     * (1 + gamma * np.random.poisson(lambda1*dt, paths)))   
        
    return Vt[:, 1:]

To find the optimal exercise time Q, first build the function that models the payment process of a loan. The loan balance is defined as $$L_t = a - bc^{12t}$$ where $a = \frac{PMT}{r}$, $b = \frac{PMT}{r(1 + r)^n}$, $c = (1 + r)$, and PMT represents monthly payment. The following function will find the value of loan given time t

In [ ]:
def loanBalance_Lt(L0, r0, T, lambda2, delta, t):
    
    # find monthly APR (r)
    R = r0 + delta*lambda2
    r = R/12
    
    # find monthly payment
    n = T * 12
    PMT = (L0*r)/(1 - (1/(1+r)**n))
    
    # Find the loan value given time t
    a = PMT/r
    b = PMT/(r*(1 + r)**n)
    c = (1 + r)
    Lt = np.clip(a - b*c**(12*t), a_max = None, a_min = 0)
    
    return Lt[1:]

Find the recovery rate of the loan at time t, denoted by $q_t$. $q_t$ is defined as $q_t = \alpha + \beta t$, where $\alpha$ is a given constant, and $\beta = \frac{\epsilon - \alpha}{T}$. We have $q_T = \epsilon$

In [ ]:
def recoveryRate_qt(alpha, eps, T, t):
    beta = (eps - alpha)/T
    qt = alpha + beta*t 
    return qt[1:]

Define exercise time $Q = min(t \geq 0: V_t \leq q_t L_t)$, which represents the first time that the relative value of the collateral crosses a threshold where it is optimal to default.The following function will find the optimal stopping time Q for every path. Set all the paths that no default happened to 5000

In [ ]:
def stopTime_Q(Vt, eps, paths, T):
    
    dt = 1/12
    t = np.arange(0, T + 0.01, 1/12)
    
    # find loan balance and recovery rate over time
    Lt = loanBalance_Lt(L0, r0, T, lambda2, delta, t)
    qt = recoveryRate_qt(alpha, eps, T, t)
    
    # find all of the time steps that the borrow are likely default, set the value of default_time at time step to 1
    residual_collateral = np.tile(Lt*qt, paths).reshape((paths, 12 * T))
    default_time = np.where(Vt - residual_collateral  <= 0, 1, 0)
    
    # Find stopping time Qt
    Q = np.argmax(default_time, axis = 1) 
    
    # Set all paths that has no default to the largest value of index
    no_default = np.where(np.sum(default_time, axis = 1) == 0)
    Q[no_default] = 5000
    
    return Q

Define another stopping time $S = min(t \leq 0: N_t > 0)$, where $N_t$ is the poisson process that represents the first default happening. Set all the paths where no default happened to 5000

In [ ]:
def stopTime_S(lambda2, paths, T):
    
    # Generate a matrix of poisson process
    dt = 1/12
    Nt = np.clip(np.random.poisson(lambda2*dt, (paths, T*12)), 
                 a_max = 1,
                 a_min = 0)
    
    S = np.argmax(Nt, axis = 1) 
    # Set all paths that has no default to the largest value of index
    no_default = np.where(np.sum(Nt, axis = 1) == 0)
    S[no_default] = 5000
    
    return S  

Now find the estimated exercise time for each path by taking the minimum of Q and S

In [ ]:
def estimated_default_time(Q, S):
    
    # find which one (Q, S) is exercised earlier
    qs = np.where(Q - S <= 0, 1, 0)
    
    # set the paths where there is no default to 5000
    no_default = np.where(Q + S == 5000*2)
    qs[no_default] = 5000
    
    return np.minimum(Q, S), qs

Find the value of default option

In [ ]:
def loanModel(V0, mu, sigma, gamma, lambda1,
                       T, paths, L0, r0, lambda2, delta, eps):
    dt = 1/12
    t = np.arange(0, T + 0.01, 1/12)
    
    # find loan collateral value
    Vt = loanCollateral_Vt(V0, mu, sigma, gamma, lambda1, T, paths)
    
    # loan balance
    Lt = np.tile(loanBalance_Lt(L0, r0, T, lambda2, delta, t), paths).reshape((paths, 12 * T))
    
    # default time Q, S
    Q = stopTime_Q(Vt, eps, paths, T)
    S = stopTime_S(lambda2, paths, T)
    
    # optimal default time
    tau, qs = estimated_default_time(Q, S)
    
    # find which type of default occured
    default_q = np.where(qs == 1)
    default_s = np.where(qs == 0)
    no_default = np.where(qs == 5000)
    
    # find discount factor of each path
    df = np.exp(-r0*dt*tau) 

    # find payoff of the default option, based on conditions
    payoff = np.zeros(paths)
    payoff[default_q] = np.maximum(Lt[default_q, tau[default_q]] 
                                   - eps* Vt[default_q, tau[default_q]], 0)

    payoff[default_s] =  np.abs(Lt[default_s, tau[default_s]] 
                                - eps* Vt[default_s, tau[default_s]])

    payoff[no_default] = 0

    # discount the expected payoff and find the value of default option 
    option_value = np.mean(payoff*df)
    
    # find default intensity
    default_intensity =  1 - len(no_default[0])/paths
    
    # find expected stopping time
    expected_stoptime = np.mean(tau[np.where(tau != 5000)[0]]*dt)
    
    d = {'Option Value': round(option_value, 4),
        'Default Intensity': round(default_intensity, 4),
        'Expected Stop Time':round(expected_stoptime, 4)}
    
    return d

Initialize parameters

In [ ]:
V0 = 20000
L0 = 22000
mu = -0.1
sigma = 0.2
gamma = -0.4
lambda1 = 0.2
T = 5
r0 = 0.02
delta = 0.25
lambda2 = 0.4
alpha = 0.7
eps = 0.95
paths = 50000
np.random.seed(7)

**(a)** Estimate the value of the default option



**(b)** Estimate the default probability


**(c)** Find the Expected Exercise Time of the default option



**Solution:**

When $\lambda_1$ = 0.2, $\lambda_2$ = 0.4, T = 5, the value, estimated default probability and expected exercise time are

In [ ]:
loan = pd.DataFrame.from_dict(loanModel(V0, mu, sigma, gamma, lambda1,
                       T, paths, L0, r0, lambda2, delta, eps), orient = 'index')
loan.columns = ['Values']
loan

**Build visualizations for different parameters**

$\lambda_1$  from 0.05 to 0.4 in increments of 0.05

$\lambda_2$ from 0.0 to 0.8 in increments of 0.1

T from 3 to 8 in increments of 1

In [ ]:
l1 = np.arange(0.05, 0.41, 0.05)
l2 = np.arange(0, 0.8, 0.1)
Time = np.arange(3, 9, 1)

**Default Option Value**

In [ ]:
lam2 = 0.4
optionValue_l1 = []
for lam1 in  l1:     
        for T in Time: 
            optionValue_l1.append(loanModel(V0, mu, sigma, gamma, lam1,
                       T, paths, L0, r0, lam2, delta, eps)['Option Value'])
optionValue_l1 = np.array(optionValue_l1).reshape((len(l1), len(Time)))

In [ ]:
lam1 = 0.2
optionValue_l2 = []
for lam2 in  l2:     
        for T in Time: 
            optionValue_l2.append(loanModel(V0, mu, sigma, gamma, lam1,
                       T, paths, L0, r0, lam2, delta, eps)['Option Value'])
optionValue_l2 = np.array(optionValue_l2).reshape((len(l2), len(Time)))

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(len(optionValue_l1)):
    plt.plot(Time, optionValue_l1[i],  linestyle='--', 
             marker='o', label = '$\lambda1 = $' + str(l1[i]))

plt.legend()
plt.xlabel('T')
plt.ylabel('Default Option Value ($)')
plt.title('Default Option Value with Different $\lambda_1$')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(len(optionValue_l2)):
    plt.plot(Time, optionValue_l2[i],  linestyle='--', 
             marker='o', label = '$\lambda2 = $' + str(l2[i]))

plt.legend()
plt.xlabel('T')
plt.ylabel('Default Option Value ($)')
plt.title('Default Option Value with Different $\lambda_2$')
plt.show()

**Default Intensity**

In [ ]:
lam2 = 0.4
defaultIntensity_l1 = []
for lam1 in  l1:     
        for T in Time: 
            defaultIntensity_l1.append(loanModel(V0, mu, sigma, gamma, lam1,
                       T, paths, L0, r0, lam2, delta, eps)['Default Intensity'])
defaultIntensity_l1 = np.array(defaultIntensity_l1).reshape((len(l1), len(Time)))

In [ ]:
lam1 = 0.2
defaultIntensity_l2 = []
for lam2 in  l2:     
        for T in Time: 
            defaultIntensity_l2.append(loanModel(V0, mu, sigma, gamma, lam1,
                       T, paths, L0, r0, lam2, delta, eps)['Default Intensity'])
defaultIntensity_l2 = np.array(defaultIntensity_l2).reshape((len(l2), len(Time)))

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(len(defaultIntensity_l1)):
    plt.plot(Time, defaultIntensity_l1[i],  linestyle='--', 
             marker='o', label = '$\lambda1 = $' + str(l1[i]))

plt.legend()
plt.xlabel('T')
plt.ylabel('Default Intensity')
plt.title('Default Intensity with Different $\lambda_1$')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(len(defaultIntensity_l2)):
    plt.plot(Time, defaultIntensity_l2[i],  linestyle='--', 
             marker='o', label = '$\lambda2 = $' + str(l2[i]))

plt.legend()
plt.xlabel('T')
plt.ylabel('Default Intensity ($)')
plt.title('Default Intensity with Different $\lambda_2$')
plt.show()

**Expected Stopping Time**

In [ ]:
lam2 = 0.4
expectedStopTime_l1 = []
for lam1 in  l1:     
        for T in Time: 
            expectedStopTime_l1.append(loanModel(V0, mu, sigma, gamma, lam1,
                       T, paths, L0, r0, lam2, delta, eps)['Expected Stop Time'])
expectedStopTime_l1 = np.array(expectedStopTime_l1).reshape((len(l1), len(Time)))

In [ ]:
lam1 = 0.2
expectedStopTime_l2 = []
for lam2 in  l2:     
        for T in Time: 
            expectedStopTime_l2.append(loanModel(V0, mu, sigma, gamma, lam1,
                       T, paths, L0, r0, lam2, delta, eps)['Expected Stop Time'])
expectedStopTime_l2 = np.array(expectedStopTime_l2).reshape((len(l2), len(Time)))

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(len(expectedStopTime_l1)):
    plt.plot(Time, expectedStopTime_l1[i],  linestyle='--', 
             marker='o', label = '$\lambda1 = $' + str(l1[i]))

plt.legend()
plt.xlabel('T')
plt.ylabel('Expected Stopping Time ($\tau$)')
plt.title('Expected Stopping Time with Different $\lambda_1$')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
for i in range(len(expectedStopTime_l2)):
    plt.plot(Time, expectedStopTime_l2[i],  linestyle='--', 
             marker='o', label = '$\lambda2 = $' + str(l2[i]))

plt.legend()
plt.xlabel('T')
plt.ylabel('Expected Stopping Time ($\tau$)')
plt.title('Expected Stopping Time with Different $\lambda_2$')
plt.show()